## Import

In [1]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

/home/aicom/.local/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.19.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
device = torch.device("cuda:4" if torch.cuda.is_available() else 'cpu')
torch.cuda.set_device(device) # change allocation of current GPU
print ('Current cuda device ', torch.cuda.current_device()) # check

Current cuda device  4


## Hyperparameter Setting

In [3]:
CFG = {
    'TRAIN_WINDOW_SIZE':120, # 90일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE': 2048,
    'SEED':73
}

In [4]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [5]:
train_data = pd.read_csv('./train.csv').drop(columns=['ID', '제품'])
sales_data = pd.read_csv('./sales.csv').drop(columns=['ID', '제품'])
brand_key_data = pd.read_csv('./brand_keyword_cnt.csv')

In [6]:
# 먼저 5번째 열부터의 데이터만 선택합니다.
data_subset = train_data.iloc[:, 5:]

# 각 행의 최댓값과 최솟값을 계산합니다.
max_values = data_subset.max(axis=1)
min_values = data_subset.min(axis=1)

# 분모가 0이 되는 경우를 처리하기 위해 조건을 적용합니다.
diff = max_values - min_values
mask = diff != 0

# 정규화 작업을 수행합니다.
train_data.loc[mask, data_subset.columns] = (data_subset[mask] - min_values[mask].values.reshape(-1, 1)) / diff[mask].values.reshape(-1, 1)
train_data.loc[~mask, data_subset.columns] = 0

# 결과를 사전에 저장합니다.
scale_max_dict = max_values.to_dict()
scale_min_dict = min_values.to_dict()

In [7]:
data_subset = sales_data.iloc[:, 5:]

# 각 행의 최댓값과 최솟값을 계산합니다.
max_values = data_subset.max(axis=1)
min_values = data_subset.min(axis=1)

# 분모가 0이 되는 경우를 처리하기 위해 조건을 적용합니다.
diff = max_values - min_values
mask = diff != 0

# 정규화 작업을 수행합니다.
sales_data.loc[mask, data_subset.columns] = (data_subset[mask] - min_values[mask].values.reshape(-1, 1)) / diff[mask].values.reshape(-1, 1)
sales_data.loc[~mask, data_subset.columns] = 0

In [8]:
data_subset = brand_key_data.iloc[:, 1:]

# 각 행의 최댓값과 최솟값을 계산합니다.
max_values = data_subset.max(axis=1)
min_values = data_subset.min(axis=1)

# 분모가 0이 되는 경우를 처리하기 위해 조건을 적용합니다.
diff = max_values - min_values
mask = diff != 0

# 정규화 작업을 수행합니다.
brand_key_data.loc[mask, data_subset.columns] = (data_subset[mask] - min_values[mask].values.reshape(-1, 1)) / diff[mask].values.reshape(-1, 1)
brand_key_data.loc[~mask, data_subset.columns] = 0

In [9]:
brand_key_data

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-04-15,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24
0,B002-00001,0.214576,0.234817,0.384615,0.655870,0.502025,0.421053,0.323886,0.307693,0.299596,...,0.105263,0.113360,0.149798,0.093119,0.137651,0.076922,0.093119,0.052632,0.060729,0.064778
1,B002-00002,0.230633,0.463479,0.312528,0.233732,0.258521,0.262948,0.208942,0.319610,0.274458,...,0.093404,0.187694,0.150066,0.149624,0.108898,0.142098,0.089863,0.100487,0.106242,0.171757
2,B002-00003,0.004359,0.006707,0.005030,0.000671,0.001677,0.007378,0.006372,0.003353,0.005365,...,0.017103,0.009725,0.015761,0.014420,0.014084,0.012072,0.002347,0.002683,0.003353,0.006372
3,B002-00004,0.008330,0.166669,0.258330,0.483334,0.375000,0.349998,0.308335,0.283333,0.241664,...,0.066665,0.000000,0.216667,0.283333,0.233334,0.333331,0.391666,0.008330,0.041663,0.308335
4,B002-00005,0.094303,0.180747,0.222004,0.208251,0.218075,0.239686,0.176818,0.135560,0.141454,...,0.216110,0.253439,0.255403,0.300590,0.239686,0.200393,0.149312,0.115914,0.229862,0.453831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3817,B002-03818,0.285702,0.321430,0.464273,0.357140,0.178570,0.553574,0.571421,0.517847,0.517847,...,0.107132,0.392851,0.285702,0.142843,0.214281,0.285702,0.107132,0.160707,0.142843,0.285702
3818,B002-03819,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3819,B002-03820,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3820,B002-03821,0.001362,0.030995,0.113533,0.070391,0.104223,0.091394,0.072321,0.077884,0.116939,...,0.023615,0.034401,0.053815,0.053020,0.035649,0.098320,0.103202,0.047798,0.043824,0.050295


In [10]:
tmp = []

for i in tqdm(train_data['브랜드']):
    idx=0
    for j in brand_key_data['브랜드']:
        if (i==j):
            tmp.append(brand_key_data.iloc[idx, :])
        idx+=1

In [11]:
brand_key_data1 = pd.DataFrame(tmp)
brand_key_data1

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-04-15,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24
0,B002-00001,0.214576,0.234817,0.384615,0.655870,0.502025,0.421053,0.323886,0.307693,0.299596,...,0.105263,0.113360,0.149798,0.093119,0.137651,0.076922,0.093119,0.052632,0.060729,0.064778
1,B002-00002,0.230633,0.463479,0.312528,0.233732,0.258521,0.262948,0.208942,0.319610,0.274458,...,0.093404,0.187694,0.150066,0.149624,0.108898,0.142098,0.089863,0.100487,0.106242,0.171757
1,B002-00002,0.230633,0.463479,0.312528,0.233732,0.258521,0.262948,0.208942,0.319610,0.274458,...,0.093404,0.187694,0.150066,0.149624,0.108898,0.142098,0.089863,0.100487,0.106242,0.171757
1,B002-00002,0.230633,0.463479,0.312528,0.233732,0.258521,0.262948,0.208942,0.319610,0.274458,...,0.093404,0.187694,0.150066,0.149624,0.108898,0.142098,0.089863,0.100487,0.106242,0.171757
1,B002-00002,0.230633,0.463479,0.312528,0.233732,0.258521,0.262948,0.208942,0.319610,0.274458,...,0.093404,0.187694,0.150066,0.149624,0.108898,0.142098,0.089863,0.100487,0.106242,0.171757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3797,B002-03798,0.138886,0.000000,0.111109,0.069438,0.111109,0.263889,0.208334,0.111109,0.097216,...,0.152770,0.000000,0.208334,0.097216,0.069438,0.069438,0.111109,0.111109,0.111109,0.180547
3798,B002-03799,0.189286,0.310715,0.385714,0.416071,0.491072,0.448214,0.342857,0.303571,0.371429,...,0.182143,0.160714,0.362500,0.382143,0.314286,0.283928,0.317858,0.044644,0.194643,0.416071
3798,B002-03799,0.189286,0.310715,0.385714,0.416071,0.491072,0.448214,0.342857,0.303571,0.371429,...,0.182143,0.160714,0.362500,0.382143,0.314286,0.283928,0.317858,0.044644,0.194643,0.416071
3798,B002-03799,0.189286,0.310715,0.385714,0.416071,0.491072,0.448214,0.342857,0.303571,0.371429,...,0.182143,0.160714,0.362500,0.382143,0.314286,0.283928,0.317858,0.044644,0.194643,0.416071


In [12]:
train_data

,대분류,중분류,소분류,브랜드,쇼핑몰,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,...,2023-04-15,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,S001-00001,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0.0,0.0,0.0,0.0,0.0,...,0.111111,0.000000,0.111111,0.000000,0.111111,0.111111,0.055556,0.000000,0.000000,0.000000
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00001,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,S001-00010,0.0,0.0,0.0,0.0,0.0,...,0.017606,0.010563,0.007042,0.007042,0.007042,0.000000,0.000000,0.000000,0.000000,0.014085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28889,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03798,S001-00001,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
28890,B002-C001-0003,B002-C002-0008,B002-C003-0042,B002-03799,S001-00001,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
28891,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,0.0,0.0,0.0,0.0,0.0,...,0.008130,0.016260,0.004065,0.000000,0.000000,0.000000,0.016260,0.000000,0.008130,0.000000
28892,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-03799,S001-00001,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
brand_key_data1

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-04-15,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24
0,B002-00001,0.214576,0.234817,0.384615,0.655870,0.502025,0.421053,0.323886,0.307693,0.299596,...,0.105263,0.113360,0.149798,0.093119,0.137651,0.076922,0.093119,0.052632,0.060729,0.064778
1,B002-00002,0.230633,0.463479,0.312528,0.233732,0.258521,0.262948,0.208942,0.319610,0.274458,...,0.093404,0.187694,0.150066,0.149624,0.108898,0.142098,0.089863,0.100487,0.106242,0.171757
1,B002-00002,0.230633,0.463479,0.312528,0.233732,0.258521,0.262948,0.208942,0.319610,0.274458,...,0.093404,0.187694,0.150066,0.149624,0.108898,0.142098,0.089863,0.100487,0.106242,0.171757
1,B002-00002,0.230633,0.463479,0.312528,0.233732,0.258521,0.262948,0.208942,0.319610,0.274458,...,0.093404,0.187694,0.150066,0.149624,0.108898,0.142098,0.089863,0.100487,0.106242,0.171757
1,B002-00002,0.230633,0.463479,0.312528,0.233732,0.258521,0.262948,0.208942,0.319610,0.274458,...,0.093404,0.187694,0.150066,0.149624,0.108898,0.142098,0.089863,0.100487,0.106242,0.171757
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3797,B002-03798,0.138886,0.000000,0.111109,0.069438,0.111109,0.263889,0.208334,0.111109,0.097216,...,0.152770,0.000000,0.208334,0.097216,0.069438,0.069438,0.111109,0.111109,0.111109,0.180547
3798,B002-03799,0.189286,0.310715,0.385714,0.416071,0.491072,0.448214,0.342857,0.303571,0.371429,...,0.182143,0.160714,0.362500,0.382143,0.314286,0.283928,0.317858,0.044644,0.194643,0.416071
3798,B002-03799,0.189286,0.310715,0.385714,0.416071,0.491072,0.448214,0.342857,0.303571,0.371429,...,0.182143,0.160714,0.362500,0.382143,0.314286,0.283928,0.317858,0.044644,0.194643,0.416071
3798,B002-03799,0.189286,0.310715,0.385714,0.416071,0.491072,0.448214,0.342857,0.303571,0.371429,...,0.182143,0.160714,0.362500,0.382143,0.314286,0.283928,0.317858,0.044644,0.194643,0.416071


In [14]:
brand_key_data1 = brand_key_data1.fillna(0)

In [15]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드', '쇼핑몰']

for col in categorical_columns:
    label_encoder.fit(sales_data[col])
    sales_data[col] = label_encoder.transform(sales_data[col])
    
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['브랜드']

for col in categorical_columns:
    label_encoder.fit(brand_key_data1[col])
    brand_key_data1[col] = label_encoder.transform(brand_key_data1[col])

# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드', '쇼핑몰']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [16]:
lists = []
for i in tqdm(range(28894)):
    pivot = np.argmax(train_data.iloc[i, 5:])
    if (pivot < 183):
        lists.append('2022-1')
    if (183<= pivot < 365):
        lists.append('2022-2')
    if (365 <= pivot):
        lists.append('2023-1')

lists = pd.DataFrame(lists, columns = ['년도'])
lists

,년도
0,2023-1
1,2022-1
2,2022-1
3,2022-1
4,2023-1
...,...
28889,2022-1
28890,2022-2
28891,2023-1
28892,2022-1


In [17]:
label_encoder = LabelEncoder()
categorical_columns = ['년도']

for col in categorical_columns:
    label_encoder.fit(lists[col])
    lists[col] = label_encoder.transform(lists[col])

In [18]:
lists

,년도
0,2
1,0
2,0
3,0
4,2
...,...
28889,0
28890,1
28891,2
28892,0


In [19]:
def make_train_data(data, data1, data2, data3,train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    num_rows = len(data)
    window_size = train_size + predict_size
    
    input_data = np.empty((num_rows * (len(data.columns) - window_size +1), train_size, len(data.iloc[0, :5]) + 3))
    target_data = np.empty((num_rows * (len(data.columns) - window_size +1), predict_size))
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :5])
        #price_data = np.array(data3.iloc[i,0])
        sales_data = np.array(data.iloc[i, 5:])
        money_data = np.array(data1.iloc[i, 5:])
        key_data = np.array(data2.iloc[i,1:])
        month_data = np.array(data3.iloc[i,:])
        
        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]
            window1 = money_data[j : j + window_size]
            window2 = key_data[j : j + window_size]
            #window3 = month_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)),np.tile(month_data, (train_size, 1)),window1[:train_size],window[:train_size]))
            #, window1[:train_size], window2[:train_size]
            input_data[i * (len(data.columns) - window_size+1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size+1) + j] = window[train_size:]
    
    return input_data, target_data

In [20]:
def make_predict_data(data, data1,data2,data3,train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
    num_rows = len(data)
    
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :5]) + 3))
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :5])
        #price_data = np.array(data3.iloc[i,0])
        sales_data = np.array(data.iloc[i, -train_size:])
        money_data = np.array(data1.iloc[i,-train_size:])
        key_data = np.array(data2.iloc[i,-train_size:])
        month_data = np.array(data3.iloc[i,-train_size:])
        
        window = sales_data[-train_size : ]
        window1 = money_data[-train_size : ]
        window2 = key_data[-train_size : ]
        #window3 = month_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)),np.tile(month_data, (train_size, 1)),window1[:train_size],window[:train_size]))
        #, window1[:train_size], window2[:train_size]
        input_data[i] = temp_data
    
    return input_data

In [21]:
train_input, train_target = make_train_data(train_data, sales_data, brand_key_data1,lists)
test_input = make_predict_data(train_data, sales_data, brand_key_data1,lists)

In [22]:
# Train / Validation Split
# Train / Validation Split
data_len = len(train_input)
val_input = train_input[-int(data_len*0.2):]
val_target = train_target[-int(data_len*0.2):]
train_input = train_input[:-int(data_len*0.2)]
train_target = train_target[:-int(data_len*0.2)]

In [23]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

((7951629, 120, 8),
 (7951629, 21),
 (1987907, 120, 8),
 (1987907, 21),
 (28894, 120, 8))

### Custom Dataset

In [24]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [25]:
train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=8)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

### 모델 선언

In [26]:
import torch
import torch.nn as nn

class DeepModel(nn.Module):
    def __init__(self, input_size=8, hidden_size=512, output_size=CFG['PREDICT_SIZE'], num_layers=3):
        super(DeepModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # LSTM layers
        self.lstm_layers = nn.ModuleList()
        self.lstm_layers.append(nn.LSTM(input_size, hidden_size, batch_first=True))
        for _ in range(num_layers - 1):
            self.lstm_layers.append(nn.LSTM(hidden_size, hidden_size, batch_first=True))
        
        # Hidden layers
        hidden_layers = []
        for _ in range(num_layers):
            hidden_layers.extend([
                nn.PReLU(),
                nn.Dropout(0.2)
            ])
        self.hidden_layer = nn.Sequential(*hidden_layers)

        # Output layer
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_size, output_size),
            nn.ReLU()
        )
        
    def forward(self, x):
        batch_size = x.size(0)
        hidden_states = self.init_hidden_states(batch_size, x.device)
        
        for i in range(self.num_layers):
            x, hidden_states[i] = self.lstm_layers[i](x, hidden_states[i])
        
        last_output = x[:, -1, :]
        hidden_output = self.hidden_layer(last_output)
        output = self.output_layer(hidden_output)
        
        return output.squeeze(1)

    def init_hidden_states(self, batch_size, device):
        hidden_states = []
        for _ in range(self.num_layers):
            hidden_states.append((
                torch.zeros(1, batch_size, self.hidden_size, device=device),
                torch.zeros(1, batch_size, self.hidden_size, device=device)
            ))
        return hidden_states


### 모델 학습

In [27]:
def train(model, optimizer, train_loader, val_loader, device):
    model = nn.DataParallel(model,device_ids = [4,5,6,7])
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None


    for epoch in range(1, 7+1):
        model.train()
        train_loss = []
        train_mae = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.float().to(device)
            Y = Y.float().to(device)

            optimizer.zero_grad()

            output = model(X)
            loss = criterion(output, Y)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')
    return best_model

In [28]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            output = model(X)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
    return np.mean(val_loss)

## Run !!

In [ ]:
model =  DeepModel()
optimizer = torch.optim.Adam(params = model.parameters(),lr = CFG["LEARNING_RATE"])
infer_model= train(model, optimizer, train_loader, val_loader, device)

## 모델 추론

In [ ]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=8)

In [ ]:
def inference(model, test_loader, device):
    predictions = []
    
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            
            output = model(X)
            
            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()
            
            predictions.extend(output)
    
    return np.array(predictions)

In [ ]:
pred = inference(infer_model, test_loader, device)

In [ ]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]
    
# 결과 후처리
pred = np.round(pred, 0).astype(int)

In [ ]:
pred.shape

## Submission

In [ ]:
submit = pd.read_csv('./sample_submission.csv')
submit.head()

In [ ]:
submit.iloc[:,1:] = pred
submit.head()

In [ ]:
submit.to_csv('./baseline_submit(lstm prelu--2022,2023,상하반기, 7).csv', index=False)